In [1]:
import logging
import warnings
logger = logging.getLogger()
logger.setLevel(logging.ERROR)
logging.basicConfig()
warnings.filterwarnings("ignore", category=UserWarning, module="pgmpy")
import random
import networkx as nx
from time import *
import matplotlib.pyplot as plt
from itertools import chain
import numpy as np
import math
import queue
from pgmpy.readwrite import BIFReader, BIFWriter

import bnlearn as bn
import pandas as pd
import matplotlib.pyplot as plt

from networkx.algorithms import connectivity
from pgmpy.factors.discrete import TabularCPD
from pgmpy.models import BayesianNetwork
from pgmpy.inference import BeliefPropagation
from pgmpy.inference import VariableElimination
from pgmpy.sampling import BayesianModelSampling
from pgmpy.estimators import BaseEstimator, MaximumLikelihoodEstimator


In [2]:
def generate_connected_dag(n, p):
    tree = nx.DiGraph()
    tree.add_nodes_from(range(n))

    for i in range(1, n):
        parent = random.randint(0, i - 1)
        tree.add_edge(parent, i)

    for i in range(n):
        for j in range(n):
            if i == j:
                continue
            if random.random() < p:
                new_tree = tree.copy()
                new_tree.add_edge(i, j)
                if nx.is_directed_acyclic_graph(new_tree):
                    tree.add_edge(i, j)

    if nx.is_directed_acyclic_graph(tree) and nx.is_weakly_connected(tree):

        mapping = {node: str(node) for node in tree.nodes}
        tree = nx.relabel_nodes(tree, mapping)
        edges_to_remove = []
        for edge in tree.edges:
            edges_to_remove.append(edge)
        for edge in edges_to_remove:
            tree.remove_edge(edge[0], edge[1])
            tree.add_edge(str(edge[0]), str(edge[1]))
        return tree 

In [3]:
def draw_dag(graph, layout_function):
    pos = layout_function(graph)
    nx.draw(graph, pos, with_labels=True, font_weight='bold', node_size=700, node_color='skyblue', arrowsize=20)
    plt.show()

In [4]:
def An(g, source): 
    G_pred = g.pred
    seen = set()
    nextlevel = source
    while nextlevel:
        thislevel = nextlevel
        nextlevel = set()
        for v in thislevel:
            if v not in seen:
                seen.add(v)
                nextlevel.update(G_pred[v])
    return seen

In [5]:
def Pass(e,u,f,v,Re,An_Re): 
    if u not in An_Re: 
        if e == 0 and f == 1:
            return False
        else:
            return True
    elif u in Re:
        if e == 0 and f == 1:
            return True
        else:
            return False
    else:
        return True

In [6]:
def Rech(g, source, Re):    
    An_Re = An(g, Re) 
    Q = []
    P = set()
    for ch in set(g.successors(source)):
        Q.append((0,ch))
        P.add((0,ch))
    for pa in set(g.predecessors(source)):
        Q.append((1,pa))
        P.add((1,pa))
    while Q:
        eV = Q.pop(0)
        for ch in set(g.successors(eV[1])):
            if (0,ch) not in P and Pass(eV[0],eV[1],0,ch,Re,An_Re):
                Q.append((0,ch))
                P.add((0,ch))
        for pa in set(g.predecessors(eV[1])):
            if (1,pa) not in P and Pass(eV[0],eV[1],1,pa,Re,An_Re):
                Q.append((1,pa))
                P.add((1,pa))
    reachable = {item[1] for item in P}
    return reachable

In [7]:
def FCMS(g,u,v):
    
    An = nx.subgraph(g, nx.ancestors(g, u) | nx.ancestors(g, v)|{u,v})
    mb_u = set([parent for child in An.successors(u) for parent in An.predecessors(child)]) | set(An.successors(u)) | set(An.predecessors(u))
    mb_u.discard(u)
    reach_v = Rech(g, v, mb_u)
    return mb_u & reach_v

In [8]:
def CMDSA(g,r):
    ang = nx.subgraph(g, An(g, r))
    h = r
    s = 1
    mark = set()
    while s:
        s = 0
        Q = set()
        m = set(g.nodes)-h
        mb = nx.node_boundary(g, m, h)
        h_ch_in_m = nx.node_boundary(g, h, m)
        for v in mb:
            pa = set(g.predecessors(v))
            Q |= (pa & h)
        for v in h_ch_in_m:
            Q |= (h & set(g.predecessors(v)))
        Q |= mb
        if len(Q)>1:
            for a in Q.copy():
                Q.remove(a)
                for b in Q:                    
                    if (a,b) not in mark and g.has_edge(a, b)==False: 
                        mark.add((a,b))
                        mark.add((b,a))
                        if g.has_edge(b,a)==False: 
                            S_a = FCMS(ang,a,b)                           
                            if not S_a:
                                continue
                            S_b = FCMS(ang,b,a)
                            if ( S_a | S_b) - (( S_a | S_b) & h):
                                s = 1
                                h |= ( S_a | S_b)  
                            break               
            else:
                continue
            break

    return h

In [9]:
def get_value(q, kwargs):
    index = []
    for var in q.variables:
        if var not in kwargs.keys():
            raise ValueError(f"Variable: {var} not found in arguments")
        else:
            try:
                index.append(q.name_to_no[var][kwargs[var]])
            except KeyError:
                logger.info(f"Using {var} state as number instead of name.")
                index.append(kwargs[var])
    return q.values[tuple(index)]

In [10]:
def generate_random_dictionary(n):
    random_dict = {}
    for i in range(n):
        random_dict[str(i)] = 2
    return random_dict

In [ ]:
dim_redu = 0

T_ve = 0
T_con = 0
T_ju = 0

TVD_ve = 0
TVD_con = 0
TVD_ju = 0

KL_ve = 0
KL_con = 0
KL_ju = 0



n=400
edges = []
for BN_num in range(0,20):
    
    nx_G = generate_connected_dag(400, 0.0012)
    
    BN = BayesianNetwork()
    BN.add_nodes_from(set(nx_G))
    BN.add_edges_from(set(nx_G.edges))
    
    state = {node: 2 for node in BN.nodes()}
    BN.get_random_cpds(n_states=state, inplace=True)
    file = f"BN_400_{BN_num}.bif"
    writer = BIFWriter(BN)
    writer.write_bif(filename=file)
    
    sampler = BayesianModelSampling(BN)
    inference_ori = VariableElimination(BN)

    
    for R_num in range(0,10): 
        R = random.sample(list(nx_G.nodes),4)
        dim_redu = dim_redu + (1 - len(CMDSA(nx_G,set(R)))/400)
        Ev = None    
        query_ori = inference_ori.query(variables = [R[0],R[1],R[2],R[3]], evidence= Ev, show_progress=False) 
        
        for i in range(0,10):          
            print(i)
            
            data = sampler.forward_sample(size=800, show_progress=False)
            
            t_s = time()
            BN_jun = BayesianNetwork()
            BN_jun.add_nodes_from(An(nx_G,R))
            BN_jun.add_edges_from(list(nx_G.subgraph(An(nx_G,R)).edges))
            sub_data = data[list(An(nx_G,R))]
            BN_jun.fit(sub_data,estimator=MaximumLikelihoodEstimator)
            inference_jun = BeliefPropagation(BN_jun)
            query_jun = inference_jun.query(variables = [R[0],R[1],R[2],R[3]], evidence= Ev, show_progress=False)
    
            for j in range(0,2):
                for k in range(0,2):
                    for l in range(0,2):
                        for t in range(0,2):
                
                            TVD_ju = TVD_ju + abs(query_jun.values[j,k,l,t] - query_ori.values[j,k,l,t])
            
                            if query_ori.values[j,k,l,t] > 0 :
                                KL_ju = KL_ju +  query_jun.values[j,k,l,t] * math.log( query_jun.values[j,k,l,t] / query_ori.values[j,k,l,t])  
            t_e = time()
            T_ju = t_e-t_s + T_ju
            
            
            t_s = time()
            BN_full = BayesianNetwork()
            BN_full = BayesianNetwork(list(nx_G.edges))
            BN_full.fit(data,estimator=MaximumLikelihoodEstimator)
        
            inference_full = VariableElimination(BN_full)
            query_full = inference_full.query(variables = [R[0],R[1],R[2],R[3]], evidence= Ev, show_progress=False)
    
            for j in range(0,2):
                for k in range(0,2):
                    for l in range(0,2):
                        for t in range(0,2):
                
                            TVD_ve = TVD_ve + abs(query_full.values[j,k,l,t] - query_ori.values[j,k,l,t])
            
                            if query_ori.values[j,k,l,t] > 0 :
                                KL_ve = KL_ve +  query_full.values[j,k,l,t] * math.log( query_full.values[j,k,l,t] / query_ori.values[j,k,l,t])  
            t_e = time()
            T_ve = t_e-t_s + T_ve
    
            #凸包
            t_s = time()
            
            BN_con = BayesianNetwork()
            H = CMDSA(nx_G,set(R))
            sub_data = data[list(H)]
            BN_con.add_nodes_from(H)
            BN_con.add_edges_from(list(nx_G.subgraph(H).edges))
    
            BN_con.fit(sub_data)
            inference_con = VariableElimination(BN_con)
            query_con = inference_con.query(variables = [R[0],R[1],R[2],R[3]], evidence= Ev, show_progress=False)
    
            for j in range(0,2):
                for k in range(0,2):
                    for l in range(0,2):
                        for t in range(0,2):
                
                            TVD_con = TVD_con + abs(query_con.values[j,k,l,t] - query_ori.values[j,k,l,t])
            
                            if query_ori.values[j,k,l,t] > 0 :
                                KL_con = KL_con +  query_con.values[j,k,l,t] * math.log( query_con.values[j,k,l,t] / query_ori.values[j,k,l,t]) 
            t_e = time()
            T_con = t_e -t_s + T_con
            
    
